In [2]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
%%time
## Loading DATA
os.chdir(r"D:\Shareholder Data\Cleaned - Merged with Price")
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
SDATA = pd.concat([pd.read_csv(f,index_col=0).drop_duplicates() for f in all_filenames ])

Wall time: 1min 50s


In [4]:
%%time
# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

Wall time: 3.53 s


In [13]:
# Filtering and keeping only the last date date.
for year in list(np.arange(1390,1399,1)):
    print(year)
    temp = SDATA[SDATA.year==year]
    last_day = temp.Jalali_Date.drop_duplicates().sort_values(ascending=True).iloc[-1]
    temp = temp[temp.Jalali_Date == last_day]

    # Saving the file
    os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
    name = 'Shareholder'+str(year)+'.csv'
    temp.to_csv(name)

1390


jdatetime.date(1390, 12, 28)

1391


jdatetime.date(1391, 12, 28)

1392


...

jdatetime.date(1396, 12, 28)

1397


jdatetime.date(1397, 12, 28)

1398


jdatetime.date(1398, 12, 28)